In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

from insync.db import ListDB
from insync.list import CompletionCommand, ListItem, ListItemContext, ListItemContextType, ListRegistry


In [3]:
descs = ['eggs', 'milk', 'bread', 'butter']

reg = ListRegistry()

for desc in descs:
    i = ListItem(desc, context=ListItemContext('grocery', ListItemContextType.CHECKLIST))
    reg.add(i)

milk_item = list(reg.items)[1]
cmd = CompletionCommand(milk_item.uuid, True)

print("\nList:\n",reg, sep='')
print("Cmd before doing:\n\t", cmd)
reg.do(cmd)
print("Cmd after doing:\n\t", cmd)
print("\nList:\n",reg, sep='')

reg.undo()
print("\nList:\n",reg, sep='')


List:
2024-03-01 eggs @^grocery
2024-03-01 milk @^grocery
2024-03-01 bread @^grocery
2024-03-01 butter @^grocery

Cmd before doing:
	 CompletionCommand(uuid=UUID('018dfd1a-eeb3-77bc-ad45-bebdc49d7585'), completed_new=True, completed_orig=None)
Cmd after doing:
	 CompletionCommand(uuid=UUID('018dfd1a-eeb3-77bc-ad45-bebdc49d7585'), completed_new=True, completed_orig=False)

List:
2024-03-01 eggs @^grocery
x 2024-03-01 milk @^grocery
2024-03-01 bread @^grocery
2024-03-01 butter @^grocery


List:
2024-03-01 eggs @^grocery
2024-03-01 milk @^grocery
2024-03-01 bread @^grocery
2024-03-01 butter @^grocery



# Save and restore from the db

In [ ]:
DB_FILE = Path("scratch.db")
DB_FILE.unlink(missing_ok=True)
db = ListDB(DB_FILE)
db.ensure_tables_created()
db.patch(reg)

In [ ]:
reg2 = db.load()

reg.add(ListItem("cheese", context=ListItemContext('grocery', ListItemContextType.CHECKLIST)))
print("\nOriginal:\n",reg, sep='')
print("\nRestored:\n",reg2, sep='')

In [41]:
import dataclasses
from typing import TypeAlias

SortKey : TypeAlias = int

@dataclasses.dataclass
class OrderableListItem:
    sort_key: SortKey
    value: int

class OrderableList:
    """test example of a list that can be ordered
    goint to benchmark the how fast the key lenth grows with the number of
    insertions and reorders
    """
    def __init__(self, MIN: SortKey, MAX: SortKey):
        self.items: list[OrderableListItem] = []

        self.MIN: SortKey = MIN
        self.MAX: SortKey = MAX

    def get_key(self, after: SortKey, before: SortKey) -> SortKey:
        """Create sort key for a new item that should be inserted between"""
        return after + (before - after) // 2

    def append(self, v: int) -> None:
        """Append an item to the end of the list"""
        after = self.MIN if len(self.items) == 0 else self.items[-1].sort_key
        key = self.get_key(after, self.MAX)
        oli = OrderableListItem(key, v)
        print(oli)
        assert key not in [i.sort_key for i in self.items], f"No space for {oli} in {self.items}"
        self.items.append(oli)

    # def insert(self, v: int, after: OrderableListItem, before: OrderableListItem) -> None:
    #     """Insert an item between two existing items"""
    #     key = self.get_key(after.sort_key, before.sort_key)
    #     self.items.append(OrderableListItem(key, v))

    @property
    def sorted_items(self) -> list[OrderableListItem]:
        return sorted(self.items, key=lambda x: x.sort_key)

    def sanity_check_insertion_order(self) -> None:
        """check that the items are in order after intial insertions"""
        from itertools import pairwise
        for a, b in pairwise(self.sorted_items):
            if a.value > b.value:
                raise ValueError(f"Items are out of order: {a} > {b}")

    def sanity_check_key_uniqueness(self) -> None:
        """check that the keys are unique"""
        keys = [i.sort_key for i in self.sorted_items]
        if len(keys) != len(set(keys)):
            raise ValueError("Keys are not unique")



In [43]:
ol = OrderableList(0, 2**8)
for j in range(6):
    ol.append(j)

ol.sanity_check_insertion_order()

import pandas as pd

df = pd.DataFrame(ol.sorted_items)

df

OrderableListItem(sort_key=128, value=0)
OrderableListItem(sort_key=192, value=1)
OrderableListItem(sort_key=224, value=2)
OrderableListItem(sort_key=240, value=3)
OrderableListItem(sort_key=248, value=4)
OrderableListItem(sort_key=252, value=5)


,sort_key,value
0,128,0
1,192,1
2,224,2
3,240,3
4,248,4
5,252,5


In [44]:
ol = OrderableList(0, 2**8)
for j in range(10):
    ol.append(j)

ol.sanity_check_insertion_order()

import pandas as pd

df = pd.DataFrame(ol.sorted_items)

df

OrderableListItem(sort_key=128, value=0)
OrderableListItem(sort_key=192, value=1)
OrderableListItem(sort_key=224, value=2)
OrderableListItem(sort_key=240, value=3)
OrderableListItem(sort_key=248, value=4)
OrderableListItem(sort_key=252, value=5)
OrderableListItem(sort_key=254, value=6)
OrderableListItem(sort_key=255, value=7)
OrderableListItem(sort_key=255, value=8)


AssertionError: No space for OrderableListItem(sort_key=255, value=8) in [OrderableListItem(sort_key=128, value=0), OrderableListItem(sort_key=192, value=1), OrderableListItem(sort_key=224, value=2), OrderableListItem(sort_key=240, value=3), OrderableListItem(sort_key=248, value=4), OrderableListItem(sort_key=252, value=5), OrderableListItem(sort_key=254, value=6), OrderableListItem(sort_key=255, value=7)]